In [1]:
import pandas as pd
import os
import numpy as np
import requests
import json
BASE_PATH = r'c:\Users\dv987\Documents\GitHub\Sample'
# Replace BASE_PATH with your own path to this folder.

In [2]:
def load_BLS(name):
    """Load and clean employment data from the Federal Bureau of Labor Statistics.

    Keyword arguments:
    name -- the name of the xlsx file
    """
    df = pd.read_excel(os.path.join(BASE_PATH, name), 
                       skiprows=2, skipfooter=5, na_values=['(n)'])
    df = df.drop([0])
    # The first row after headers is empy so we drop it.
    df = df.drop(['LAUS Code','ST FIPS Code'], axis=1)
    df = df.rename(columns={'Area':'MSA', 'Area FIPS Code':'FIPS Code'})
    df = df.dropna(how='all')
    # We'll later be merging this dataframe with one that measures time in years.
    # To solve this, we'll agreggate this data by year now by averaging months together.
    df_aggregated = df.drop('Month', axis=1).groupby(['MSA', 'Year']).mean().reset_index()
    return df_aggregated
BLS = load_BLS('ssamatab1.xlsx')
BLS.head()

,MSA,Year,FIPS Code,Civilian Labor Force,Employment,Unemployment,Unemployment Rate
0,"Abilene, TX MSA",1990.0,10180.0,68508.250000,64433.583333,4074.666667,5.950000
1,"Abilene, TX MSA",1991.0,10180.0,69379.333333,65679.000000,3700.333333,5.325000
2,"Abilene, TX MSA",1992.0,10180.0,71042.083333,67115.083333,3927.000000,5.525000
3,"Abilene, TX MSA",1993.0,10180.0,72597.166667,68664.250000,3932.916667,5.408333
4,"Abilene, TX MSA",1994.0,10180.0,73926.916667,70252.583333,3674.333333,4.958333


In [7]:
def load_BEA():
    """ Load and clean manufacturing employment data from the Bureau of Economic Analysis using its API."""
    userid = '6633B66D-A212-4FF2-B8AE-46E60688BD22'
    method = 'GetData'
    dataset = 'Regional'
    results = 'json'
    # These parameters indicate the dataset we want and the format of the data.
    tablename = 'CAEMP25N'
    linecode = '500'
    geofips = 'MSA'
    year = 'ALL'
    # These dataset-specific parameters indicate which table and column we retrieve, and the geographic resolution of the data. 
    url = f'https://apps.bea.gov/api/data/?UserID={userid}&method={method}&datasetname={dataset}&TableName={tablename}&LineCode={linecode}&GeoFips={geofips}&Year={year}&ResultFormat={results}'
    response = requests.get(url).json()
    df = pd.json_normalize(response['BEAAPI']['Results']['Data'])
    df.to_csv('nipa_T10101.csv', index=False)
    df = df.drop(['Code', 'UNIT_MULT', 'NoteRef', 'CL_UNIT'], axis=1)
    df = df.rename(columns={'GeoName':'MSA', 'GeoFips':'FIPS Code', 'TimePeriod':'Year', 'DataValue':'Manufacturing Jobs'})
    df['FIPS Code'] = df['FIPS Code'].astype(int)
    df['Year'] = df['Year'].astype(int)
    # We'll need FIPS Code and Year to be integers to properly merge the dataframes.
    return df
BEA = load_BEA()
BEA.head()

,FIPS Code,MSA,Year,Manufacturing Jobs
0,998,United States (Metropolitan Portion),2001,13722906
1,998,United States (Metropolitan Portion),2002,12728492
2,998,United States (Metropolitan Portion),2003,12080129
3,998,United States (Metropolitan Portion),2004,11906135
4,998,United States (Metropolitan Portion),2005,11838580


In [6]:
# Now that both datasets are loaded and cleaned, we can merge them by FIPS code.
def merge_BLS_BEA(bls, bea):
    """Merge the BLS and BEA dataframes."""
    df_merged = bls.merge(bea, on=('FIPS Code', 'Year'), how='inner', indicator=True)
    df_merged = df_merged.drop(['MSA_y', '_merge'], axis=1)
    df_merged = df_merged.rename(columns={'MSA_x':'MSA'})
    df_merged['FIPS Code'] = df_merged['FIPS Code'].astype(int)
    df_merged['Year'] = df_merged['Year'].astype(int)
    return df_merged
merged = merge_BLS_BEA(BLS, BEA)
merged.head()

,MSA,Year,FIPS Code,Civilian Labor Force,Employment,Unemployment,Unemployment Rate,Manufacturing Jobs
0,"Abilene, TX MSA",2001,10180,75389.583333,72030.083333,3359.500000,4.458333,3593
1,"Abilene, TX MSA",2002,10180,77672.833333,73736.583333,3936.250000,5.066667,3479
2,"Abilene, TX MSA",2003,10180,79534.750000,75280.833333,4253.916667,5.350000,3163
3,"Abilene, TX MSA",2004,10180,79591.166667,75845.250000,3745.916667,4.716667,3385
4,"Abilene, TX MSA",2005,10180,80444.833333,77011.083333,3433.750000,4.266667,3478
